## Install required libraries


In [ ]:
pip install -q tensorflow tensorflow-datasets

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np


from tensorflow import keras
tfds.disable_progress_bar()

## Tensorflow Dataset library 

In [ ]:
tfds.list_builders()

In [ ]:
builder = tfds.builder('rock_paper_scissors')
info = builder.info
print(info)

tfds.core.DatasetInfo(
    name='rock_paper_scissors',
    version=3.0.0,
    description='Images of hands playing rock, paper, scissor game.',
    homepage='http://laurencemoroney.com/rock-paper-scissors-dataset',
    features=FeaturesDict({
        'image': Image(shape=(300, 300, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=3),
    }),
    total_num_examples=2892,
    splits={
        'test': 372,
        'train': 2520,
    },
    supervised_keys=('image', 'label'),
    citation="""@ONLINE {rps,
    author = "Laurence Moroney",
    title = "Rock, Paper, Scissors Dataset",
    month = "feb",
    year = "2019",
    url = "http://laurencemoroney.com/rock-paper-scissors-dataset"
    }""",
    redistribution_info=,
)



## Load Data

In [ ]:
ds_train = tfds.load(name="rock_paper_scissors", split="train")
ds_test = tfds.load(name="rock_paper_scissors", split="test")

In [ ]:
tfds.show_examples(ds_train, info)

## Preparation of Data

In [ ]:
train_images = np.array([example["image"].numpy()[:,:,0] for example in ds_train])
train_labels = np.array([example["label"].numpy()for example in ds_train])

test_images = np.array([example["image"].numpy()[:,:,0] for example in ds_test])
test_labels = np.array([example["label"].numpy()for example in ds_test])

In [ ]:
test_images.shape

(372, 300, 300)

In [ ]:
train_images = train_images.reshape(2520, 300, 300, 1)
test_images = test_images.reshape(372, 300, 300, 1)

train_images = train_images.astype('float32')
test_images = test_images.astype('float32')

train_images /= 255
test_images /= 255

## Building up the basic model

1.   Flatten: Eliminate the dimension of input array and make it 1 dimension array.
2.   Dense: Dense layer is a connection layer between input and output. It takes info from previous layer and send it to next layer.





In [ ]:
model = keras.Sequential([
  keras.layers.Flatten(),
  keras.layers.Dense(512, activation='relu'),
  keras.layers.Dense(256, activation='relu'),
  keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer = 'adam',
              loss = keras.losses.SparseCategoricalCrossentropy(),
              metrics = ['accuracy'])

model.fit(train_images, train_labels, epochs = 5, batch_size = 32)

Epoch 1/5
79/79 [==============================] - 3s 28ms/step - loss: 23.4429 - accuracy: 0.4087
Epoch 2/5
79/79 [==============================] - 2s 26ms/step - loss: 2.3114 - accuracy: 0.6298
Epoch 3/5
79/79 [==============================] - 2s 25ms/step - loss: 1.3290 - accuracy: 0.7377
Epoch 4/5
79/79 [==============================] - 2s 26ms/step - loss: 0.3881 - accuracy: 0.8587
Epoch 5/5
79/79 [==============================] - 2s 26ms/step - loss: 0.2986 - accuracy: 0.8944


In [ ]:
model.evaluate(test_images, test_labels)

12/12 [==============================] - 0s 12ms/step - loss: 2.1611 - accuracy: 0.4651


[2.161128520965576, 0.4650537669658661]

## Adding Convolutional Layers

This layer creates a convolution kernel that is convolved with the layer input to produce a tensor of outputs



```
tf.keras.layers.Conv2D(
    filters,
    kernel_size,
    strides=(1, 1),
    padding="valid",
    data_format=None,
    dilation_rate=(1, 1),
    groups=1,
    activation=None,
    use_bias=True,
    kernel_initializer="glorot_uniform",
    bias_initializer="zeros",
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,
    **kwargs
)

```




In [ ]:
model = keras.Sequential([
  keras.layers.Conv2D(64, 3, activation='relu', input_shape=(300,300,1)),
  keras.layers.Conv2D(32, 3, activation='relu'),
  keras.layers.Flatten(),
  keras.layers.Dense(3, activation='softmax')

])

model.compile(optimizer = 'adam',
              loss = keras.losses.SparseCategoricalCrossentropy(),
              metrics = ['accuracy'])

model.fit(train_images, train_labels, epochs = 5, batch_size = 32)

Epoch 1/5
79/79 [==============================] - 26s 319ms/step - loss: 4.4217 - accuracy: 0.6429
Epoch 2/5
79/79 [==============================] - 25s 319ms/step - loss: 0.5102 - accuracy: 0.8873
Epoch 3/5
79/79 [==============================] - 25s 321ms/step - loss: 0.1133 - accuracy: 0.9810
Epoch 4/5
79/79 [==============================] - 25s 320ms/step - loss: 0.0366 - accuracy: 0.9940
Epoch 5/5
79/79 [==============================] - 25s 321ms/step - loss: 0.0165 - accuracy: 0.9980


In [ ]:
model.evaluate(test_images, test_labels)

12/12 [==============================] - 3s 228ms/step - loss: 1.4123 - accuracy: 0.5941


[1.4122563600540161, 0.5940860509872437]

## Adding Average Pooling for extract features more smoothly

In [ ]:
model = keras.Sequential([
  keras.layers.AveragePooling2D(6, 3, input_shape=(300,300,1)),
  keras.layers.Conv2D(64, 3, activation='relu'),
  keras.layers.Conv2D(32, 3, activation='relu'),
  keras.layers.MaxPool2D(2,2),
  keras.layers.Dropout(0.5),
  keras.layers.Flatten(),
  keras.layers.Dense(3, activation='softmax')

])

model.compile(optimizer = 'adam',
              loss = keras.losses.SparseCategoricalCrossentropy(),
              metrics = ['accuracy'])

model.fit(train_images, train_labels, epochs = 5, batch_size = 32)

Epoch 1/5
79/79 [==============================] - 31s 47ms/step - loss: 0.8851 - accuracy: 0.6341
Epoch 2/5
79/79 [==============================] - 3s 43ms/step - loss: 0.2189 - accuracy: 0.9560
Epoch 3/5
79/79 [==============================] - 3s 43ms/step - loss: 0.0661 - accuracy: 0.9853
Epoch 4/5
79/79 [==============================] - 3s 43ms/step - loss: 0.0306 - accuracy: 0.9940
Epoch 5/5
79/79 [==============================] - 3s 43ms/step - loss: 0.0162 - accuracy: 0.9972


In [ ]:
model.evaluate(test_images, test_labels)

12/12 [==============================] - 0s 24ms/step - loss: 0.7930 - accuracy: 0.7124


[0.7930178642272949, 0.7123655676841736]

## Keras Tuner for Hyperparamater Optimization

In [ ]:
pip install keras-tuner 

In [ ]:
import keras_tuner as kt
from keras_tuner import RandomSearch


def build_model(hp):
  model = keras.Sequential()

  model.add(keras.layers.AveragePooling2D(6,3,input_shape=(300,300,1)))

  for i in range(hp.Int("Conv Layers", min_value=0, max_value=3)):
    model.add(keras.layers.Conv2D(hp.Choice(f"layer_{i}_filters", [16,32,64]), 3, activation='relu'))
  
  model.add(keras.layers.MaxPool2D(2,2))
  model.add(keras.layers.Dropout(0.5))
  model.add(keras.layers.Flatten())

  model.add(keras.layers.Dense(hp.Choice("Dense layer", [64, 128, 256, 512, 1024]), activation='relu'))

  model.add(keras.layers.Dense(3, activation='softmax'))

  model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
  
  return model


tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=32,
)

tuner.search(train_images, train_labels, validation_data=(test_images, test_labels), epochs=10, batch_size=32)

Trial 27 Complete [00h 01m 12s]
val_accuracy: 0.7580645084381104

Best val_accuracy So Far: 0.8037634491920471
Total elapsed time: 00h 17m 58s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [ ]:
best_model = tuner.get_best_models()[0]

best_model.evaluate(test_images, test_labels)

12/12 [==============================] - 0s 22ms/step - loss: 0.7430 - accuracy: 0.8038


[0.7429729700088501, 0.8037634491920471]

In [ ]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
average_pooling2d (AveragePo (None, 99, 99, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 97, 97, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 95, 95, 16)        2320      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 93, 93, 16)        2320      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 46, 46, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 46, 46, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 33856)             0

In [ ]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
Conv Layers: 2
Dense layer: 256
layer_0_filters: 16
layer_1_filters: 64
layer_2_filters: 64
Score: 0.8037634491920471
Trial summary
Hyperparameters:
Conv Layers: 1
Dense layer: 128
layer_0_filters: 64
layer_1_filters: 64
layer_2_filters: 32
Score: 0.7849462628364563
Trial summary
Hyperparameters:
Conv Layers: 3
Dense layer: 64
layer_0_filters: 32
layer_1_filters: 16
layer_2_filters: 32
Score: 0.7795698642730713
Trial summary
Hyperparameters:
Conv Layers: 1
Dense layer: 64
layer_0_filters: 16
layer_1_filters: 64
layer_2_filters: 64
Score: 0.7768816947937012
Trial summary
Hyperparameters:
Conv Layers: 1
Dense layer: 128
layer_0_filters: 64
layer_1_filters: 32
layer_2_filters: 16
Score: 0.7715053558349609
Trial summary
Hyperparameters:
Conv Layers: 2
Dense layer: 256
layer_0_filters: 32
layer_1_filters: 64
layer_2_filters: 64
Score: 0.76881718

## Videos For CNN Visualizing
https://www.youtube.com/watch?v=pj9-rr1wDhM

